# Factor models and expected returns

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pandas_datareader import DataReader as pdr
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px


Pull factor data

In [2]:
# Pull data
ff3 = pdr("F-F_Research_Data_Factors", "famafrench", start=1926)
ff5 = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=1964)

ff3_monthly = ff3[0]/100
ff5_monthly = ff5[0]/100

ff3_annual  = ff3[1]/100
ff5_annual  = ff5[1]/100

C:\Users\kpc2\projects\ipykernel_2596\275179538.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3 = pdr("F-F_Research_Data_Factors", "famafrench", start=1926)
C:\Users\kpc2\projects\ipykernel_2596\275179538.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3 = pdr("F-F_Research_Data_Factors", "famafrench", start=1926)
C:\Users\kpc2\projects\ipykernel_2596\275179538.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5 = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=1964)
C:\Users\kpc2\projects\ipykernel_2

Estimate factor risk premia as time-series average return (using annual data)

In [3]:
# Annual 3 factors from 1926
fprem = ff3_annual[["Mkt-RF", "SMB", "HML"]].mean()

# Add annual RNW and CMA factors from 1964
fprem = pd.concat((fprem, ff5_annual[["RMW", "CMA"]].mean()))
factors = fprem.index.to_list()
fprem = fprem.round(4)
fprem

Mkt-RF    0.0880
SMB       0.0270
HML       0.0412
RMW       0.0362
CMA       0.0313
dtype: float64

Estimate factor loadings

In [4]:
# Function to fetch a yahoo time-series
def returns(ticker):
    ret = yf.download(ticker, start='2000-01-01', end='2024-12-31', progress=False)
    ret = ret["Close"].resample("ME").last()
    ret = ret.pct_change()
    ret.index = ret.index.to_period('M')
    ret.columns = ['ret']
    return ret

In [5]:
# Pull stock returns
TICKER = 'IBM'
ret = returns('IBM')

YF.download() has changed argument auto_adjust default to True


In [6]:
yf.download('IBM', start='2000-01-01', end='2024-12-31', progress=False)

Price,Close,High,Low,Open,Volume
Ticker,IBM,IBM,IBM,IBM,IBM
Date,,,,,
2000-01-03,58.507248,58.507248,56.426713,56.710420,10823694
2000-01-04,56.521278,57.750688,55.922336,57.498501,8606279
2000-01-05,58.507248,60.398648,56.552807,56.962607,13318927
2000-01-06,57.498486,59.988824,57.246299,59.515975,8338607
2000-01-07,57.246304,59.484455,55.796232,59.137700,12402108
...,...,...,...,...,...
2024-12-23,220.461243,222.259277,219.616877,221.335424,2988100
2024-12-24,222.924835,222.954635,220.073819,220.798999,1186200


In [7]:
# Add in factor returns and risk-free rate
df = pd.merge(ret,ff5_monthly, how="left",left_index=True,right_index=True)
df = df.dropna()
df

,ret,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,
2000-02,-0.083669,0.0245,0.1828,-0.0959,-0.1865,-0.0048,0.0043
2000-03,0.152068,0.0520,-0.1532,0.0813,0.1179,-0.0159,0.0047
2000-04,-0.058078,-0.0640,-0.0501,0.0726,0.0766,0.0565,0.0046
2000-05,-0.036395,-0.0442,-0.0381,0.0475,0.0413,0.0137,0.0050
2000-06,0.020967,0.0464,0.0992,-0.0842,-0.0831,-0.0295,0.0040
...,...,...,...,...,...,...,...
2024-08,0.061194,0.0161,-0.0365,-0.0113,0.0085,0.0086,0.0048
2024-09,0.093752,0.0174,-0.0102,-0.0259,0.0004,-0.0026,0.0040
2024-10,-0.064954,-0.0097,-0.0088,0.0089,-0.0138,0.0103,0.0039


In [ ]:
# Calculate excess return
df['xret'] = df.ret - df.RF 

In [9]:
# Estimate factor loadings using last 60 months
result = sm.OLS(df.ret.iloc[-60:], sm.add_constant(df[factors].iloc[-60:])).fit()
betas = result.params[1:]
betas = np.round(betas, 2)
betas

Mkt-RF    0.74
SMB      -0.17
HML       0.26
RMW       0.22
CMA       0.32
dtype: float64

Calculate expected return using 
1. estimated factor loadings (`betas`)
2. estimated factor risk premia (`fprem`)
3. risk-free rate (`rf`)

In [10]:
# Most recent risk-free rate (in decimal notation)
rf = pdr("DGS3MO", "fred", start=1920).iloc[-1]/100
rf = rf.iloc[0]
rf

0.0433

In [11]:
# Expected return
expret = rf + betas @ fprem
print(f'The estimated expected return for {TICKER} is:\t {expret: .1%}')

The estimated expected return for IBM is:	  13.3%
